### homework 2-1: Phoneme Classification

### download data
download data from [kaggle](https://www.kaggle.com/c/ml2021spring-hw2/data) <br>
`timit_11/`
- `train_11.npy`: training data<br>
- `train_label_11.npy`: training label<br>
- `test_11.npy`:  testing data<br><br>

### preparing Data
load the training and testing data from the `.npy` file (numpy array)

In [ ]:
import numpy as np

print('loading data ...')

data_root = './timit_11/'
train = np.load(data_root + 'train_11.npy')
train_label = np.load(data_root + 'train_label_11.npy')
test = np.load(data_root + 'test_11.npy')

print('size of training data: {}'.format(train.shape))
print('size of testing data: {}'.format(test.shape))

### create dataset

In [ ]:
import torch
from torch.utils.data import Dataset

class TIMITDataset(Dataset):
  def __init__(self, X, y=None):
    self.data = torch.from_numpy(X).float()
    if y is not None:
      y = y.astype(np.int64)
      self.label = torch.LongTensor(y)
    else:
      self.label = None

  def __getitem__(self, idx):
    if self.label is not None:
      return self.data[idx], self.label[idx]
    else:
      return self.data[idx]

  def __len__(self):
    return len(self.data)


split the labeled data into a training set and a validation set, <br>
you can modify the variable VAL_RATIO to change the ratio of validation data

In [ ]:
VAL_RATIO = 0.2

percent = int(train.shape[0] * (1 - VAL_RATIO))
train_x, train_y, val_x, val_y = train[:percent], train_label[:percent], train[percent:], train_label[percent:]
print('size of training set: {}'.format(train_x.shape))
print('size of validation set: {}'.format(val_x.shape))

create a data loader from the dataset, feel free to tweak the variable BATCH_SIZE here

In [ ]:
BATCH_SIZE = 64

from torch.utils.data import DataLoader

train_set = TIMITDataset(train_x, train_y)
val_set = TIMITDataset(val_x, val_y)
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True) #only shuffle the training data
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False)

cleanup the unneeded variables to save memory.<br>

**notes: if you need to use these variables later, <br>
then you may remove this block or clean up unneeded variables later<br>
the data size is quite huge, so be aware of memory usage in colab**

In [ ]:
import gc

del train, train_label, train_x, train_y, val_x, val_y
gc.collect()

### create model

define model architecture, you are encouraged to change and experiment with the model architecture.

In [ ]:
import torch
import torch.nn as nn

class Classifier(nn.Module):
  def __init__(self):
    super().__init__()
    self.layer1 = nn.Linear(429, 1024)
    self.layer2 = nn.Linear(1024, 512)
    self.layer3 = nn.Linear(512, 128)
    self.out = nn.Linear(128, 39)

    self.act_fn = nn.Sigmoid()

  def forward(self, x):
    x = self.layer1(x)
    x = self.act_fn(x)

    x = self.layer2(x)
    x = self.act_fn(x)

    x = self.layer3(x)
    x = self.act_fn(x)

    x = self.out(x)
    
    return x

### training

In [ ]:
# check device
def get_device():
  return 'cuda' if torch.cuda.is_available() else 'cpu'

fix random seeds for reproducibility

In [ ]:
# fix random seed
def same_seeds(seed):
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  
  np.random.seed(seed)  
  torch.backends.cudnn.benchmark = False
  torch.backends.cudnn.deterministic = True

feel free to change the training parameters here

In [ ]:
# fix random seed for reproducibility
same_seeds(0)

# get device 
device = get_device()
print(f'DEVICE: {device}')

# training parameters
num_epoch = 20                # number of training epoch
learning_rate = 0.0001       # learning rate

# the path where checkpoint saved
model_path = './model.ckpt'

# create model, define a loss function, and optimizer
model = Classifier().to(device)
criterion = nn.CrossEntropyLoss() 
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# start training

best_acc = 0.0
for epoch in range(num_epoch):
  train_acc = 0.0
  train_loss = 0.0
  val_acc = 0.0
  val_loss = 0.0

  # training
  model.train()  # set the model to training mode
  for i, data in enumerate(train_loader):
    inputs, labels = data
    inputs, labels = inputs.to(device), labels.to(device)
    optimizer.zero_grad()
    outputs = model(inputs)
    batch_loss = criterion(outputs, labels)
    print(f"outputs.shape {outputs.shape} labels.shape {labels.shape} batch_loss.shape {batch_loss.shape}")
    _, train_pred = torch.max(outputs, 1)  # get the index of the class with the highest probability
    batch_loss.backward()
    optimizer.step()

    train_acc += (train_pred.cpu() == labels.cpu()).sum().item()
    train_loss += batch_loss.item()

  # validation
  if len(val_set) > 0:
    model.eval()  # set the model to evaluation mode
    with torch.no_grad():
      for i, data in enumerate(val_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        batch_loss = criterion(outputs, labels)
        _, val_pred = torch.max(outputs, 1)

        val_acc += (val_pred.cpu() == labels.cpu()).sum().item()  # get the index of the class with the highest probability
        val_loss += batch_loss.item()

      print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f} | Val Acc: {:3.6f} loss: {:3.6f}'.format(
          epoch + 1, num_epoch, train_acc / len(train_set), train_loss / len(train_loader), val_acc / len(val_set), val_loss / len(val_loader)
      ))

      # if the model improves, save a checkpoint at this epoch
      if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), model_path)
        print('saving model with acc {:.3f}'.format(best_acc / len(val_set)))
  else:
    print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f}'.format(
        epoch + 1, num_epoch, train_acc / len(train_set), train_loss / len(train_loader)
    ))

# if not validating, save the last epoch
if len(val_set) == 0:
  torch.save(model.state_dict(), model_path)
  print('saving model at last epoch')


### testing
create a testing dataset, and load model from the saved checkpoint

In [ ]:
# create testing dataset
test_set = TIMITDataset(test, None)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)

# create model and load weights from checkpoint
model = Classifier().to(device)
model.load_state_dict(torch.load(model_path))

make prediction

In [ ]:
predict = []
model.eval()  # set the model to evaluation mode
with torch.no_grad():
  for i, data in enumerate(test_loader):
    inputs = data
    inputs = inputs.to(device)
    outputs = model(inputs)
    _, test_pred = torch.max(outputs, 1)  # get the index of the class with the highest probability

    for y in test_pred.cpu().numpy():
      predict.append(y)

write prediction to a CSV file

after finish running this block, download the file pred.csv from the files section on the left-hand side and submit it to [Kaggle](https://www.kaggle.com/c/ml2021spring-hw2/)

In [ ]:
file = 'pred.csv'
print('saving results to {}'.format(file))
with open(file, 'w') as f:
  f.write('id,class\n')
  for i, y in enumerate(predict):
    f.write('{},{}\n'.format(i, y))